# Function Calling (Fonksiyon Çağırma)

## 🎯 Bu Kodun Amacı
AI'nın kendi kararıyla Python fonksiyonlarını çağırabilmesini sağlamak.

## Ne Yapacağız?
- Python'da veri analizi fonksiyonu (`get_numeric_summary`) yazacağız
- Bu fonksiyonu **tools** parametresi ile AI'ya tanıtacağız
- AI, kullanıcı sorusuna göre hangi fonksiyonu çağıracağına **otomatik karar** verecek
- AI'nın **tool_calls** yanıtını kontrol edip fonksiyonu çalıştıracağız
- Fonksiyon sonucunu AI'ya geri göndereceğiz ve final cevabı alacağız

## Çalıştırma
Kod çalıştırıldığında AI otomatik olarak veri analiz fonksiyonunu çağırır ve sonuçları yorumlar.

In [1]:
import os
from dotenv import load_dotenv
from groq import Groq
import pandas as pd
import json

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

df = pd.read_csv('airline_passenger_satisfaction.csv')

## 🔧 Veri Analiz Fonksiyonu Tanımlama

**Ne yapıyoruz?**
- `get_numeric_summary()`: Numerik kolonların istatistiklerini hesaplayan bir fonksiyon
- AI bu fonksiyonu otomatik olarak çağırabilecek

**Fonksiyon ne yapıyor?**
- `select_dtypes(include=['number'])`: Sadece numerik kolonları seçer
- İlk 5 numerik kolon için: `mean` (ortalama), `max` (maksimum), `min` (minimum) hesaplar
- Sonucu dictionary olarak döndürür

**Neden fonksiyon?**
- AI'nın kendi başına yapamayacağı işlemler (veri okuma, hesaplama)
- Gerçek zamanlı veri erişimi
- Güvenli ve kontrollü kod çalıştırma

In [2]:
def get_numeric_summary():
    """Numerik kolonların özet istatistiklerini döndürür"""
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    summary = {}
    for col in numeric_cols[:5]:
        summary[col] = {
            "mean": float(round(df[col].mean(), 2)),
            "max": float(df[col].max()),
            "min": float(df[col].min())
        }
    return summary

## 🛠️ Fonksiyonu AI'ya Tanıtma ve İlk Çağrı

**Ne yapıyoruz?**
- Fonksiyonumuzu `tools` parametresi ile AI'ya tanıtıyoruz
- AI, kullanıcı sorusuna göre bu fonksiyonu çağırıp çağırmayacağına karar verir

**tools listesi:**
- `type: "function"`: Bu bir fonksiyon tanımı
- `name`: Fonksiyonun adı (Python'daki fonksiyon adıyla aynı olmalı)
- `description`: AI'nın ne zaman kullanacağını anlaması için açıklama
- `parameters`: Fonksiyonun aldığı parametreler (bu örnekte yok)

**tool_choice="auto":**
- AI otomatik karar verir
- Alternatifler: "none" (hiç çağırma), "required" (mutlaka çağır)

**message değişkeni:**
- AI'nın ilk yanıtını kaydediyoruz
- Eğer fonksiyon çağrısı yaptıysa `message.tool_calls` dolu olacak

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_numeric_summary",
            "description": "Veri setindeki numerik kolonların istatistiklerini verir",
            "parameters": {"type": "object", "properties": {}}
        }
    }
]

user_question = "Bu veri setindeki numerik değerlerin ortalamalarını göster"

response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": user_question}],
    tools=tools,
    tool_choice="auto"
)

message = response.choices[0].message

## 🚀 Fonksiyonu Çalıştırma ve Final Cevap Alma

**Ne yapıyoruz?**
- AI'nın fonksiyon çağırıp çağırmadığını kontrol ediyoruz
- Çağırdıysa fonksiyonu çalıştırıp sonucu AI'ya geri gönderiyoruz

**if message.tool_calls:**
- AI fonksiyon çağırdıysa True
- `get_numeric_summary()` çalıştırılır ve sonuç alınır

**İkinci API çağrısı:**
- 3 mesaj gönderiyoruz:
  1. Orijinal kullanıcı sorusu
  2. AI'nın ilk yanıtı (fonksiyon çağrısı)
  3. Fonksiyon sonucu (`role: "tool"`)

**tool_call_id:**
- AI'nın hangi fonksiyon çağrısının sonucunu gönderdiğimizi anlaması için
- Her fonksiyon çağrısının benzersiz bir ID'si var

**Final cevap:**
- AI fonksiyon sonucunu görüp kullanıcıya anlaşılır şekilde yorumlar
- "Ortalamalar: ID: 64940.5, Age: 39.43..." gibi

**Function Calling akışı:**
```
1. Kullanıcı sorar: "Ortalamaları göster"
2. AI: "Bu iş için get_numeric_summary fonksiyonunu çağırmalıyım"
3. Biz: Fonksiyonu çalıştırırız → Sonuç: {ID: {mean: 64940.5, ...}, ...}
4. AI'ya sonucu göndeririz
5. AI: "İşte ortalamalar: ID: 64940.5, Age: 39.43..."
```

In [4]:
if message.tool_calls:
    function_result = get_numeric_summary()
    print(f"Fonksiyon sonucu:\n{json.dumps(function_result, indent=2)}\n")
    
    final_response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "user", "content": user_question},
            message,
            {
                "role": "tool",
                "tool_call_id": message.tool_calls[0].id,
                "content": json.dumps(function_result)
            }
        ]
    )
    
    print("FINAL CEVAP:")
    print(final_response.choices[0].message.content)

Fonksiyon sonucu:
{
  "ID": {
    "mean": 64940.5,
    "max": 129880.0,
    "min": 1.0
  },
  "Age": {
    "mean": 39.43,
    "max": 85.0,
    "min": 7.0
  },
  "Flight Distance": {
    "mean": 1190.32,
    "max": 4983.0,
    "min": 31.0
  },
  "Departure Delay": {
    "mean": 14.71,
    "max": 1592.0,
    "min": 0.0
  },
  "Arrival Delay": {
    "mean": 15.09,
    "max": 1584.0,
    "min": 0.0
  }
}

FINAL CEVAP:
Bu veri setindeki numerik değerlerin ortalamaları şu şekilde:

- ID: 64940.5
- Age: 39.43
- Flight Distance: 1190.32
- Departure Delay: 14.71
- Arrival Delay: 15.09

Bu ortalamalar, veri setindeki her bir numerik değişkenin ortalama değerini göstermektedir.
